<a href="https://colab.research.google.com/github/alexmijo/6.806-Homeworks/blob/main/6864_hw1_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import itertools as it
import numpy as np
np.random.seed(0)

import lab_util

## Hidden Markov Models

In the remaining part of the lab (containing part 3) you'll use the Baum--Welch algorithm to learn _categorical_ representations of words in your vocabulary. Answers to questions in this lab should go in the same report as the initial release.

As before, we'll start by loading up a dataset:

In [ ]:
data = []
n_positive = 0
n_disp = 0
with open("reviews.csv") as reader:
  csvreader = csv.reader(reader)
  next(csvreader)
  for id, review, label in csvreader:
    label = int(label)

    # hacky class balancing
    if label == 1:
      if n_positive == 2000:
        continue
      n_positive += 1
    if len(data) == 4000:
      break

    data.append((review, label))
    
    if n_disp > 5:
      continue
    n_disp += 1
    print("review:", review)
    print("rating:", label, "(good)" if label == 1 else "(bad)")
    print()

print(f"Read {len(data)} total reviews.")
np.random.shuffle(data)
reviews, labels = zip(*data)
train_reviews = reviews[:3000]
train_labels = labels[:3000]
val_reviews = reviews[3000:3500]
val_labels = labels[3000:3500]
test_reviews = reviews[3500:]
test_labels = labels[3500:]

review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
rating: 1 (good)

review: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
rating: 0 (bad)

review: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother an

Next, implement the forward--backward algorithm for HMMs like we saw in class.

**IMPORTANT NOTE**: if you directly multiply probabilities as shown on the class slides, you'll get underflow errors. You'll probably want to work in the log domain (remember that `log(ab) = log(a) + log(b)`, `log(exp(a) + exp(b)) = logaddexp(a, b)`). In general, we recommend either `np.logaddexp` or `scipy.special.logsumexp` as safe ways to compute the necessary quantities.

In [ ]:
from scipy import special

# hmm model
class HMM(object):
    def __init__(self, num_states, num_words):
        self.num_states = num_states
        self.num_words = num_words

        self.states = range(num_states)
        self.symbols = range(num_words)

        """
        Initialize the matrix A with random transition probabilities p(j|i)
        A should be a matrix of size `num_states x num_states` with rows that
        sum to 1.
        """
        self.A = np.random.rand(num_states, num_states)
        self.A = self.A / self.A.sum(axis=1, keepdims=True)

        """
        Initialize the matrix B with random emission probabilities p(o|i). B 
        should be a matrix of size `num_states x num_words` with rows that sum 
        to 1.
        """
        self.B = np.random.rand(num_states, num_words)
        self.B = self.B / self.B.sum(axis=1, keepdims=True)

        """
        Initialize the vector pi with a random starting distribution. pi should
        be a vector of size `num_states` with entries that sum to 1.
        """
        self.pi = np.random.rand(num_states)
        self.pi = self.pi / self.pi.sum()

    def generate(self, n):
        """randomly sample the HMM to generate a sequence.
        """
        # we'll give you this one

        sequence = []
        # initialize the first state
        state = np.random.choice(self.states, p=self.pi)
        for i in range(n):
            # get the emission probs for this state
            b = self.B[state, :]
            # emit a word
            word = np.random.choice(self.symbols, p=b)
            sequence.append(word)
            # get the transition probs for this state
            a = self.A[state, :]
            # update the state
            state = np.random.choice(self.states, p=a)
        return sequence

    def forward(self, obs):
        """
        Runs the forward algorithm. This function should return a 
        `len(obs) x  num_states` matrix where the (t, i)th entry contains 
        log p(obs[:t], hidden_state_t = i)
        """

        alpha = np.zeros((len(obs), self.num_states))
        logA = np.log(self.A)
        logB = np.log(self.B)
        logPi = np.log(self.pi)

        alpha[0] = logPi + logB[:, obs[0]]

        for t in range(len(obs) - 1):
            alpha[t + 1] = scipy.special.logsumexp(alpha[t][:, None] + logA,\
                                                   axis=0) + logB[:, obs[t + 1]]

        return alpha

    def backward(self, obs):
        """
        Run the backward algorithm. This function should return a
        `len(obs) x num_states` matrix where the (t, i)th entry contains
        log p(obs[t+1:] | hidden_state_t = i)
        """

        beta = np.zeros((len(obs), self.num_states))
        logA = np.log(self.A)
        logB = np.log(self.B)

        for t in range(len(obs) - 2, -1, -1):
            beta[t] = scipy.special.logsumexp(logA + logB[:, obs[t + 1]] +\
                                              beta[t + 1], axis=1)

        return beta
        
    def forward_backward(self, obs):
        """
        Compute forward-backward scores

        logprob is the total log-probability of the sequence obs (marginalizing
        over hidden states).

        gamma is a matrix of size `len(obs) x num_states1. It contains the
        marginal probability of being in state i at time t

        xi is a tensor of size `len(obs) - 1 x num_states x num_states`. It contains
        the marginal probability of transitioning from i to j at t.
        """

        alpha = self.forward(obs)
        beta = self.backward(obs)
        logA = np.log(self.A)
        logB = np.log(self.B)

        logprob = scipy.special.logsumexp(alpha[-1])
        xi = np.zeros((len(obs) - 1, self.num_states, self.num_states))
        for t in range(len(obs) - 1):
            xi[t] = alpha[t][:, None] + logA + logB[:, obs[t + 1]] +\
             beta[t + 1] - logprob
        gamma = alpha + beta - logprob

        return logprob, np.exp(xi), np.exp(gamma)

        """
        SANITY CHECK

        The most straightforward way of implementing the forward, backward, and 
        forward_backward methods would be to iterate through all the values and 
        use the formulas in the slides to calculate the corresponding values.

        However, this may not be fast enough. If your model is taking too long
        to train, consider how you may speed up your code by reducing the number
        of for loops involved. How can you reformulate your code using matrix
        operations?

        Hint: we were able to implement each of the forward, backward, and
        forward_backward operations using only one for loop.
        """

    def learn_unsupervised(self, corpus, num_iters, print_every=10):
        """Run the Baum Welch EM algorithm
        
        corpus: the data to learn from
        num_iters: the number of iterations to run the algorithm
        print_every: how often to print the log-likelihood while the model is
        updating its parameters.
        """

        for i_iter in range(num_iters):
            """
            expected_si: a vector of size (num_states,) where the i-th entry is
            the expected number of times a sentence is transitioning from state 
            i to some other state.

            expected_sij: an array of size (num_states, num_states) where the
            (i,j)-th entry represents the expected number of state transitions
            between state i and state j.

            expected_sjwk: an array of size (num_states, num_words) where the 
            (j,k)-th entry represents the expected number of times the word w_k 
            appears when at state j.

            expected_sj: a vector of size (num_states,) where the j-th entry is
            the expected number of times a sentence is in state j (differing 
            from expected_si by also including end states).

            expected_q1: a vector of size (num_states,) where the i-th entry is 
            the expected number of times state i is the first state.

            total_logprob: The log of the probability of the corpus being
            generated with the current parameters of the HMM.
            """
            expected_si = np.zeros((self.num_states,))
            expected_sij = np.zeros((self.num_states, self.num_states))
            expected_sjwk = np.zeros((self.num_states, self.num_words))
            expected_sj = np.zeros((self.num_states,))
            expected_q1 = np.zeros((self.num_states))
            total_logprob = 0
            
            for review in corpus:
                logprob, xi, gamma = self.forward_backward(review)
                expected_si += np.sum(gamma[0:-1], axis=0)
                expected_sij += np.sum(xi, axis=0)
                for t in range(len(review)):
                    k = review[t]
                    expected_sjwk[:,k] += gamma[t]
                expected_sj += np.sum(gamma, axis=0)
                expected_q1 += gamma[0]
                total_logprob += logprob
            expected_q1 = expected_q1 / np.sum(expected_q1)
            if i_iter % print_every == 0:
              print("log-likelihood", total_logprob)

            """
            The following variables should be the new values of self.A, self.B,
            and self.pi after the values are updated.
            """
            A_new = expected_sij / expected_si[:, None]
            B_new = expected_sjwk / expected_sj[:, None]
            pi_new = expected_q1

            self.A = A_new
            self.B = B_new
            self.pi = pi_new

## Test Cases

The following are test cases that are meant to help you debug your code. The code involves six test suites - an initialization test, a forward test, a backward test, a forward_backward test, a baum_welch_update test, and a final end_to_end test.

In [ ]:
def init_test():

    num_states = np.random.randint(100)
    num_words = np.random.randint(100)
    model = HMM(num_states, num_words)

    assert model.A.shape == (num_states, num_states)
    assert model.B.shape == (num_states, num_words)
    assert model.pi.shape == (num_states, )

    assert np.linalg.norm(np.sum(model.A, axis=1) - np.ones(num_states)) < 1e-10
    assert np.linalg.norm(np.sum(model.B, axis=1) - np.ones(num_states)) < 1e-10
    assert np.linalg.norm(np.sum(model.pi) - 1) < 1e-10

def forward_test():
    model = HMM(2, 10)
    model.A = np.array([[0.79034887, 0.20965113],
                        [0.66824331, 0.33175669]])
    model.B = np.array([[0.08511814, 0.06627238, 0.08487461, 0.15607959, 0.00124582, 0.12984083, 0.11164849, 0.11591902, 0.15232716, 0.09667395],
                        [0.18425462, 0.14326559, 0.14026994, 0.0215989,  0.17687124, 0.04681278, 0.05857451, 0.17451212, 0.00473382, 0.04910648]])
    model.pi = np.array([0.77480039, 0.22519961])
    obs = [1, 8, 0, 0, 3, 4, 5, 2, 6, 3, 7, 9]
    alpha = model.forward(obs)

    print("The result of the forward function should be", np.array([[-2.96913, -3.43382],
                                                                    [ -4.66005, -9.19418],
                                                                    [ -7.35001, -7.89695],
                                                                    [ -9.65069, -9.95363],
                                                                    [-11.25815, -14.27392],
                                                                    [-18.14079, -14.4781 ],
                                                                    [-16.89275, -18.62696],
                                                                    [-19.45549, -20.17289],
                                                                    [-21.53772, -23.283  ],
                                                                    [-23.4927, -26.69119],
                                                                    [-25.84891, -26.73817],
                                                                    [-28.12237, -29.92402]]))
    print("Your value of alpha is:", np.round(alpha, 5))

def backward_test():
    model = HMM(2, 10)
    model.A = np.array([[0.79034887, 0.20965113],
                        [0.66824331, 0.33175669]])
    model.B = np.array([[0.08511814, 0.06627238, 0.08487461, 0.15607959, 0.00124582, 0.12984083, 0.11164849, 0.11591902, 0.15232716, 0.09667395],
                        [0.18425462, 0.14326559, 0.14026994, 0.0215989,  0.17687124, 0.04681278, 0.05857451, 0.17451212, 0.00473382, 0.04910648]])
    model.pi = np.array([0.77480039, 0.22519961])
    obs = [1, 8, 0, 0, 3, 4, 5, 2, 6, 3, 7, 9]
    beta = model.backward(obs)

    print("The result of the backward function should be", np.array([[-25.42937, -25.58918], 
                                                                     [-23.32164, -23.19959],
                                                                     [-21.11007, -21.02033],
                                                                     [-18.82215, -18.94381],
                                                                     [-16.78523, -16.33951],
                                                                     [-13.42847, -13.51924],
                                                                     [-11.24815, -11.19161],
                                                                     [ -8.88679,  -8.96441],
                                                                     [ -6.57374,  -6.70985],
                                                                     [ -4.51873,  -4.47419],
                                                                     [ -2.44529,  -2.51463],
                                                                     [  0, 0]]))

    print("Your value of beta is:", np.round(beta, 5))


def forward_backward_test():
    model = HMM(2, 10)
    model.A = np.array([[0.79034887, 0.20965113],
                        [0.66824331, 0.33175669]])
    model.B = np.array([[0.08511814, 0.06627238, 0.08487461, 0.15607959, 0.00124582, 0.12984083, 0.11164849, 0.11591902, 0.15232716, 0.09667395],
                        [0.18425462, 0.14326559, 0.14026994, 0.0215989,  0.17687124, 0.04681278, 0.05857451, 0.17451212, 0.00473382, 0.04910648]])
    model.pi = np.array([0.77480039, 0.22519961])
    obs = [1, 8, 0, 0, 3, 4, 5, 2, 6, 3, 7, 9]
    logprob, xi, gamma = model.forward_backward(obs)

    print("The value of logprob should be:", -27.9693)
    print("Your value of logprob is:", np.round(logprob, 5))

    print("The value of xi should be:", np.array([[[0.64523, 0.00601],
                                                  [0.34278, 0.00598]],

                                                 [[0.60684, 0.38117],
                                                  [0.00551, 0.00648]],

                                                 [[0.40595, 0.2064 ],
                                                  [0.19863, 0.18902]],

                                                 [[0.5718,  0.03278],
                                                  [0.35711, 0.03831]],

                                                 [[0.02625, 0.90266],
                                                  [0.00109, 0.07   ]],

                                                 [[0.02482, 0.00251],
                                                  [0.81777, 0.15489]],

                                                 [[0.59943, 0.24316],
                                                  [0.08947, 0.06793]],

                                                 [[0.6143,  0.07461],
                                                  [0.25347, 0.05762]],

                                                 [[0.8357,  0.03207],
                                                  [0.12337, 0.00886]],

                                                 [[0.69872, 0.26034],
                                                  [0.02412, 0.01682]],

                                                 [[0.63701, 0.08583],
                                                  [0.22134, 0.05582]]]))
    print("Your value of xi is:", np.round(xi, 5))

    print("The value of gamma should be:", np.array([[0.65124, 0.34876],
                                                    [0.98802, 0.01198],
                                                    [0.61235, 0.38765],
                                                    [0.60458, 0.39542],
                                                    [0.92891, 0.07109],
                                                    [0.02733, 0.97267],
                                                    [0.8426,  0.1574 ],
                                                    [0.68891, 0.31109],
                                                    [0.86777, 0.13223],
                                                    [0.95906, 0.04094],
                                                    [0.72284, 0.27716],
                                                    [0.85835, 0.14165]]))

    print("Your value of gamma is:", np.round(gamma, 5))

def baum_welch_update_test():
    model = HMM(4, 10)
    
    model.A = np.array([[0.05263151, 0.62161178, 0.06683182, 0.25892489],
                        [0.26993274, 0.13114741, 0.32305468, 0.27586517],
                        [0.2951958,  0.14576492, 0.22474111, 0.33429817],
                        [0.29586018, 0.26065884, 0.1977772,  0.24570378]])
    
    model.B = np.array([[0.01800425, 0.09767131, 0.17824799, 0.12586453, 0.19514548, 0.05433139, 0.01995667, 0.12985343, 0.01884263, 0.16208232],
                        [0.04512782, 0.09469685, 0.1426164,  0.13851362, 0.08717793, 0.17152532, 0.08746939, 0.04900339, 0.05315859, 0.13071069],
                        [0.11055806, 0.10592473, 0.0051817,  0.07721441, 0.21761783, 0.20323146, 0.18881598, 0.00584989, 0.00682669, 0.07877924],
                        [0.08711377, 0.16703645, 0.0706214,  0.05297571, 0.10486868, 0.16794587, 0.13562053, 0.15729142, 0.03345308, 0.02307309]])
    
    model.pi = np.array([0.21186864, 0.27156561, 0.37188523, 0.14468051])
    
    corpus = np.array([[7,3,2,5,0,3,2,9,4,2], [7,3,2,4,2,8,7,5,0,8], [7,3,2,3,1,7,3,8,6,7], [7,3,2,6,4,4,3,4,0,0]])

    model.learn_unsupervised(corpus, 200)

    print("hmm.A should be", np.array([[0, 1, 0, 0], 
                                     [0.14122, 0, 0.27099, 0.58779], 
                                     [0.20671, 0, 0, 0.79329], 
                                     [0, 0.90909, 0.09091, 0]]))
    print("Your implementation has hmm.A to be", np.round(model.A, 5))

    print("hmm.B should be", np.array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                                              [0.0625, 0, 0, 0.5, 0, 0.125, 0.125, 0, 0.125, 0.0625],
                                              [0, 0.20671, 0, 0, 0.79329, 0, 0, 0, 0, 0],
                                              [0.24667, 0, 0.57555, 0, 0.09556, 0, 0, 0, 0.08222, 0]]))
    print("Your implementation has hmm.B to be", np.round(model.B, 5))

    print("hmm.pi should be", np.array([1, 0, 0, 0]))

    print("Your implementation has hmm.pi to be", np.round(model.pi, 5))

def end_to_end_test():
    # Test Case 1

    corpus = np.array([[0,3,0,3,0,3,0,3,0,3,0,3], [0,2,0,2,0,2,0,2,0,2,0,2,0], [1,2,1,2,1,2,1,2,1,2,1,2],[1,3,1,3,1,3,1,3,1,3]])
    hmm = HMM(num_states=2,num_words=4)
    hmm.learn_unsupervised(corpus, 10)
    print("After this test case, hmm.A should either be approximately,",  np.array([[0, 1], [1, 0]]))
    print("This is your current value of hmm.A: ", np.round(hmm.A, 5))

    print("After this test case, hmm.B should either be approximately,", np.array([[0, 0, 0.5, 0.5], [0.5, 0.5, 0, 0]]), " or it should be ", np.array([[0.5, 0.5, 0, 0], [0, 0, 0.5, 0.5]]))
    print("This is your current value of hmm.B: ", np.round(hmm.B, 5))

    # Test Case 2

    corpus = np.array([[0,0,0,0,0,0,0,0,0,0], [1,1,1,1,1,1,1,1,1,1], [2,2,2,2,2,2,2,2,2,2]])
    hmm = HMM(num_states=3, num_words=3)
    hmm.learn_unsupervised(corpus, 100)
    print("After this test case, hmm.A should be the identity matrix", np.eye(3))
    print("This is your current value of hmm.A: ", np.round(hmm.A, 5))

    print("After this test case, hmm.B should be some 3 by 3 permutation matrix")
    print("This is your current value of hmm.B: ", np.round(hmm.B, 5))

## Test

To actually run the test cases, run the cell below:

In [ ]:
init_test()
forward_test()
backward_test()
forward_backward_test()
baum_welch_update_test()
end_to_end_test()

"""
Note: The end_to_end_test is not as robustg due to it using random starts. Try
running the test case a few times to see if you get a good result at least a few
times before deciding that your code is buggy.
"""

The result of the forward function should be [[ -2.96913  -3.43382]
 [ -4.66005  -9.19418]
 [ -7.35001  -7.89695]
 [ -9.65069  -9.95363]
 [-11.25815 -14.27392]
 [-18.14079 -14.4781 ]
 [-16.89275 -18.62696]
 [-19.45549 -20.17289]
 [-21.53772 -23.283  ]
 [-23.4927  -26.69119]
 [-25.84891 -26.73817]
 [-28.12237 -29.92402]]
Your value of alpha is: [[ -2.96913  -3.43382]
 [ -4.66005  -9.19418]
 [ -7.35001  -7.89695]
 [ -9.65069  -9.95363]
 [-11.25815 -14.27392]
 [-18.14079 -14.4781 ]
 [-16.89275 -18.62697]
 [-19.45549 -20.17289]
 [-21.53772 -23.283  ]
 [-23.4927  -26.69119]
 [-25.84891 -26.73817]
 [-28.12237 -29.92402]]
The result of the backward function should be [[-25.42937 -25.58918]
 [-23.32164 -23.19959]
 [-21.11007 -21.02033]
 [-18.82215 -18.94381]
 [-16.78523 -16.33951]
 [-13.42847 -13.51924]
 [-11.24815 -11.19161]
 [ -8.88679  -8.96441]
 [ -6.57374  -6.70985]
 [ -4.51873  -4.47419]
 [ -2.44529  -2.51463]
 [  0.        0.     ]]
Your value of beta is: [[-25.42937 -25.58918]
 [-23.32

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in log


log-likelihood -58.78006168233149
log-likelihood -58.056386229045785
log-likelihood -58.047573998672505
log-likelihood -58.04756024248872
log-likelihood -58.04756014666196
log-likelihood -58.04756014303576
log-likelihood -58.04756014156317
log-likelihood -58.047560140773356
log-likelihood -58.04756014034711
log-likelihood -58.047560140117014
log-likelihood -58.04756013999281
log-likelihood -58.047560139925785
log-likelihood -58.0475601398896
log-likelihood -58.04756013987008
log-likelihood -58.04756013985953
log-likelihood -58.04756013985383
log-likelihood -58.04756013985075
hmm.A should be [[0.      1.      0.      0.     ]
 [0.14122 0.      0.27099 0.58779]
 [0.20671 0.      0.      0.79329]
 [0.      0.90909 0.09091 0.     ]]
Your implementation has hmm.A to be [[0.      1.      0.      0.     ]
 [0.14122 0.      0.27099 0.58779]
 [0.20671 0.      0.      0.79329]
 [0.      0.90909 0.09091 0.     ]]
hmm.B should be [[0.      0.      0.      0.      0.      0.      0.      1.      0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
log-likelihood -3.295836866004329
After this test case, hmm.A should be the identity matrix [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
This is your current value of hmm.A:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
After this test case, hmm.B should be some 3 by 3 permutation matrix
This is your current value of hmm.B:  [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


'\nNote: The end_to_end_test is not as robustg due to it using random starts. Try\nrunning the test case a few times to see if you get a good result at least a few\ntimes before deciding that your code is buggy.\n'

## Training

Train a model:

In [ ]:
tokenizer = lab_util.Tokenizer()
tokenizer.fit(train_reviews)
train_reviews_tk = tokenizer.tokenize(train_reviews)
print(tokenizer.vocab_size)

hmm = HMM(num_states=10, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, 10)

2006
log-likelihood -2074409.1474417406


Let's look at some of the words associated with each hidden state:

In [ ]:
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[-10:]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

state 0
and 0.016854516606141096
is 0.017724502965300922
. 0.026651384300688905
of 0.02694921618751614
the 0.027825377348346086
for 0.029802677540205522
in 0.03227147728361361
a 0.056296898544462376
<unk> 0.07534971276229671
, 0.07812662306778717

state 1
is 0.011921706915693707
are 0.01268285492414466
we 0.01840940369773672
these 0.026211606072903668
they 0.02751933152885832
you 0.03281521312655056
br 0.04791835071092612
the 0.06341391656783049
this 0.08719760773475778
i 0.2269775969525636

state 2
not 0.011473574600711111
them 0.013462193786072219
a 0.016357856134287602
in 0.018014227756421218
. 0.024086117056006832
it 0.03812742306574631
, 0.04838843703935807
is 0.050440639736794454
to 0.06817857354812536
<unk> 0.10679692000362417

state 3
amazon 0.010151497979271083
taste 0.010222165046040755
br 0.011636099259617023
are 0.01373023219161479
, 0.013896582188709882
i 0.01663176637915253
a 0.019814527105397238
<unk> 0.042785985730371554
. 0.04500384904417197
the 0.04820132835574991

st

We can also look at some samples from the model!

In [ ]:
for i in range(10):
    print(tokenizer.de_tokenize([hmm.generate(10)]))

['yuck personally you are it soda sound cake too all']
['purchased br br but are being <unk> <unk> <unk> trying']
['in i in my flavoring . brand paying beans design']
['this have hot . not of with on this it']
['ever lunch but br as low much so but water']
['are or . <unk> <unk> day more orange taste for']
['only need of across to , the . it flavor']
['<unk> wrong problem good not the this i wanted ,']
["it's night lunch my for different to . grocery that"]
['i egg , like br in the just will <unk>']


Finally, let's repeat the classification experiment from Parts 1 and 2, using the _vector of expected hidden state counts_ as a sentence representation.

(Warning! results may not be the same as in earlier versions of this experiment.)

In [ ]:
def train_model(xs_featurized, ys):
  import sklearn.linear_model
  model = sklearn.linear_model.LogisticRegression(max_iter=1000)
  model.fit(xs_featurized, ys)
  return model

def eval_model(model, xs_featurized, ys):
  pred_ys = model.predict(xs_featurized)
  print("test accuracy", np.mean(pred_ys == ys))

def training_experiment(name, featurizer, n_train):
    print(f"{name} features, {n_train} examples")
    train_xs = np.array([
        hmm_featurizer(review) 
        for review in tokenizer.tokenize(train_reviews[:n_train])
    ])
    train_ys = train_labels[:n_train]
    test_xs = np.array([
        hmm_featurizer(review)
        for review in tokenizer.tokenize(test_reviews)
    ])
    test_ys = test_labels
    model = train_model(train_xs, train_ys)
    eval_model(model, test_xs, test_ys)
    print()

def hmm_featurizer(review):
    _, _, gamma = hmm.forward_backward(review)
    return gamma.sum(axis=0)

training_experiment("hmm", hmm_featurizer, n_train=3000)

hmm features, 3000 examples
test accuracy 0.704



## Experiments for Part 3

In [ ]:
# Part 3 (b)
tokenizer = lab_util.Tokenizer()
tokenizer.fit(train_reviews)
train_reviews_tk = tokenizer.tokenize(train_reviews)
print(tokenizer.vocab_size)

hmm = HMM(num_states=2, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, 10)
print("2 states:\n")
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[-100:]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

hmm = HMM(num_states=10, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, 10)
print("10 states:\n")
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[-100:]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

hmm = HMM(num_states=100, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, 10)
print("100 states:\n")
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[-15:]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

# Expiriments for Part 3 (c) were simply done in the above
# code cell by changing n_train

2006
log-likelihood -2103005.673700632
2 states:

state 0
real 0.001454877095314458
store 0.0014571677142538333
use 0.0014611493576798084
she 0.001485778678345754
which 0.0014986301325533983
been 0.0015035094766905775
me 0.0015208517252846992
about 0.0015320444220478723
give 0.0015383977846400955
think 0.0015403772999669594
your 0.0015523744896875817
sugar 0.0015835191246662231
amazon 0.0015960533966783569
salt 0.0016214953420510866
on 0.0016448366563066891
don't 0.0016956978202699224
want 0.001763551110159697
only 0.0017969965734653827
there 0.0018018435976250309
found 0.001807324298922035
good 0.0018170967129075065
even 0.0018244918800645947
got 0.001850141417173182
drink 0.0018553857754342253
make 0.001881190736204208
great 0.0019006004054356728
bought 0.001917242871366712
try 0.001940594786815735
can 0.001994880330177523
what 0.0020356787507069944
say 0.002047346579252729
less 0.0020850185885147037
small 0.00208988484917833
something 0.0020928264370810535
buy 0.0022203249539339952
